In [1]:
#This lab uses the hitters.csv file to demonstrate forward selection, backward elimination, stepwise selection
#and different validation methods. Lab tasks done in both R and Python
import rpy2.rinterface
%load_ext rpy2.ipython
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor

##In Python
##Read in the data
hitters = pd.read_csv('hitters.csv')
n = len(hitters['Salary'])
##a
expVars = hitters.drop(labels = "Salary", axis = 1)
expVars = "+".join(expVars.columns)

y, X = dmatrices('Salary ~' + expVars, hitters, return_type='dataframe')
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["variable"] = X.columns
vif

,VIF Factor,variable
0,21.762082,Intercept
1,4.134115,League[T.N]
2,1.075398,Division[T.W]
3,4.099063,NewLeague[T.N]
4,22.944366,AtBat
5,30.281255,Hits
6,7.758668,HmRun
7,15.246418,Runs
8,11.921715,RBI
9,4.148712,Walks


In [2]:
%%R
##In R
##Read in data and load libraries
library(MASS)
#library(ISLR)
#library(boot)
library(car)
hitters <- read.csv(file = "hitters.csv", header = T)
n <- dim(hitters)[1]

##a
mod <- lm('Salary ~ .', data = hitters)
vif(mod)


Error in library(car) : there is no package called ‘car’


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in library(car) : there is no package called ‘car’

  warnings.warn(x, RRuntimeWarning)


Multicollinearity does appear to be an issue. We typically point out variables with VIF >5 or so, and there are several variables in this data set with larger VIF than 5 or even 10. The three variables with the largest VIFs are Career Hits ($CHits = 502.954$), Career At Bats ($CAtBat = 251.56116$), and Career Runs ($CRuns = 162.5208$). It's not really surprising that these are all very high, because they are likely dependent upon one another. A player can't have a lot of hits if he doesn't have very many at bats, and hits mean the player gets on base, which increases the chances that he will score.

In [3]:
%%R
##b
library(leaps)
all_poss <- regsubsets(Salary ~ ., data = hitters, nvmax = 19)
all_poss_summ <- summary(all_poss)
all_poss_summ


Error in library(leaps) : there is no package called ‘leaps’


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in library(leaps) : there is no package called ‘leaps’

  warnings.warn(x, RRuntimeWarning)


In [4]:
%%R
max_idx <- which.max(all_poss_summ$adjr2)
max_idx


Error in which.max(all_poss_summ$adjr2) : 
  object 'all_poss_summ' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in which.max(all_poss_summ$adjr2) : 
  object 'all_poss_summ' not found

  warnings.warn(x, RRuntimeWarning)


In [5]:
%%R
plot(all_poss_summ$adjr2, type = "l", xlab = "Number of Explanatory Variables", ylab = bquote("Adjusted R"^2), main = "Optimal Number of Explanatory Variables")
points(all_poss_summ$adjr2, pch = 16)
points(x = max_idx, y = all_poss_summ$adjr2[max_idx], pch = 16, col = "red")


Error in plot(all_poss_summ$adjr2, type = "l", xlab = "Number of Explanatory Variables",  : 
  object 'all_poss_summ' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in plot(all_poss_summ$adjr2, type = "l", xlab = "Number of Explanatory Variables",  : 
  object 'all_poss_summ' not found

  warnings.warn(x, RRuntimeWarning)


In [6]:
%%R
##This tells us we want 11 variables in the model.. so which are they?
all_poss_summ$which[max_idx,]


Error in withVisible({ : object 'all_poss_summ' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in withVisible({ : object 'all_poss_summ' not found

  warnings.warn(x, RRuntimeWarning)


We see here that using 11 explanatory variables gives us the optimal model for this data set. We will want to use the variables AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWalks, League, Division, PutOuts, and Assists.

For each possible number of explanatory variables (in this case, for 0 explanatory variables up to all 19), All Possible Regressions fits every possible model (determined by the unique combinations of $x_i$'s) and decides which new variable in each number category (as the number increases) is the most significant and therefore most important to include in the model for that specific number of variables. For all of these models, R stores the selection criteria BIC, $R_{adj}^2$, and $C_p$ (which we haven't talked about but is in the book). Then, the user picks one of those and selects the optimal number of variables according to that selection criterion. For example, in what I did above, I used $R_{adj}^2$ and had R tell me what number of variables had the highest $R_{adj}^2$ and which of those variables gave me that $R_{adj}^2$.

In [7]:
%%R
##c
library(MASS)
smallest <- lm(Salary ~ 1, data = hitters)
largest <- lm(Salary ~ ., data = hitters)

# Forward Selection
stepAIC(object = smallest, scope = list(upper = largest, lower = smallest), direction = "forward")
mod_forward <- stepAIC(object = smallest, scope = list(upper = largest, lower = smallest), direction = "forward", trace = 0)


Error in is.data.frame(data) : object 'hitters' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in is.data.frame(data) : object 'hitters' not found

  warnings.warn(x, RRuntimeWarning)


In [8]:
%%R
summary(mod_forward)


Error in summary(mod_forward) : object 'mod_forward' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in summary(mod_forward) : object 'mod_forward' not found

  warnings.warn(x, RRuntimeWarning)


Using forward selection, R identified the best model as the one with the 10 variables AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWalks, PutOuts, Assists, and Division.

Forward selection begins by fitting the smallest model possible-- the one with no variables (intercept only). Calculate its AIC. Then, we calculate the AIC for each of the 1 variable models; if the AIC for a 1 variable model is smaller than that of the intercept only model, then we add the 1 variable model with the smallerst AIC. Otherwise, we stick with the current intercept only model. Next, we consider the 2 variable models that have our first chosen variable as one of the two. If the AIC would improve by adding another variable to the model (as indicated by a smaller AIC), then we add the variable that shrinks the AIC the most. Otherwise, we stick with the 1 variable model. This continues until we reach a point where either a) there are no variables we could add that would improve/reduce the AIC or b) we have added all variables to the model. 

So in this specific case, we started with an AIC of 3215.77 and first added CRBI (which reduced the AIC to 3115.8), then Hits (reducing AIC to 3074.1), then PutOuts, etc. up until after we added Assists, which is when the AIC reached a value (3031.26) that would not be reduced upon adding any of the remaining variables.

In [9]:
%%R
#d
# Backward elimination
stepAIC(object = largest, scope = list(upper = largest, lower = smallest), direction = "backward")
mod_backward <- stepAIC(object = largest, scope = list(upper = largest, lower = smallest), direction = "backward", trace = 0)


Error in terms(object) : object 'largest' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in terms(object) : object 'largest' not found

  warnings.warn(x, RRuntimeWarning)


In [10]:
%%R
summary(mod_backward)


Error in summary(mod_backward) : object 'mod_backward' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in summary(mod_backward) : object 'mod_backward' not found

  warnings.warn(x, RRuntimeWarning)


With backward elimination, the model with the 10 variables AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWalks, PutOuts, Assists, and Division was also deemed the best model.

In backward elimination, we begin by fitting the largest model possible-- the full model. Calculate its AIC. Then, we calculate the AIC for each of the $n-1 = 18$ variable models; if the AIC for an 18 variable model is smaller than that of the full model, then we take the 18 variable model without the variable that, upon removal, lowers the AIC the most. Otherwise, we stick with the current full model. Next, we consider the $n-2 = 17$ variable models that have our first chosen variable as one of the two. If the AIC would decrease by dropping another variable to the model, then we drop the variable that shrinks the AIC the most. Otherwise, we stick with the 18 variable model. This continues until we reach a model where either a) there are no variables we could remove that would improve/reduce the AIC or b) we have eliminated all variables from the model. 

So in this specific case, we started with an AIC of 3046.02 and first dropped CHmRun (which reduced the AIC to 3044.03), then dropped Years (reducing AIC to 3042.1), then New League, etc. up until after we eliminated League, which is when the AIC reached a value (3031.26) that would not be reduced upon getting rid any of the remaining variables.

In [11]:
%%R
#e
# Hybrid
stepAIC(object = smallest, scope = list(upper = largest, lower = smallest), direction = "both")
mod_hybrid <- stepAIC(object = smallest, scope = list(upper = largest, lower = smallest), direction = "both", trace = 0)


Error in terms(object) : object 'smallest' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in terms(object) : object 'smallest' not found

  warnings.warn(x, RRuntimeWarning)


In [12]:
%%R
summary(mod_hybrid)


Error in summary(mod_hybrid) : object 'mod_hybrid' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in summary(mod_hybrid) : object 'mod_hybrid' not found

  warnings.warn(x, RRuntimeWarning)


As with the previous two stepwise selection techniques, hybrid selection decided the model with the 10 variables AtBat, Hits, Walks, CAtBat, CRuns, CRBI, CWalks, PutOuts, Assists, and Division was the best.

Hybrid selection begins as Forward Selection does-- by fitting the intercept only model, calculating the AIC, and considering all 1 variable models. If the AIC for a 1 variable model is smaller than that of the intercept only model, then we take the 1 variable model with the variable for which the AIC will be the smallest. Otherwise, we stick with the intercept only model. Next, we consider all 2 variable models that have our first chosen variable as one of the two. If the AIC would improve by adding another variable to the model (as indicated by a smaller AIC), then we add the variable that shrinks the AIC the most. Otherwise, we stick with the 1 variable model. (R also considers dropping the first variable, which is redundant, as if this would lower AIC, we would not have added the variable in the first place.) Then, we consider all 3 variable models with the first two chosen variables and an additional one. If the AIC would decrease with another variable added, then we take that three variable model with the smallest AIC. At this step and for the remaining steps, R considers the AICs for all models with one additional variable and one fewer variable (taking all combinations). This allows for the possibility of getting rid of a variable that would be important if we were going to stick to a fixed, small-variable model but may not be important as other variable interactions are added to the model. This continues until we reach a point where  there are no variables we could add or drop that would improve/reduce the AIC.

So in this specific case, we again started with an AIC of 3215.77 and first added CRBI (which reduced the AIC to 3115.8), then Hits (reducing AIC to 3074.1), then PutOuts, etc. up until after we added Assists, which is when the AIC reached a value (3031.26) that would not be reduced upon adding any of the remaining variables. Note that in this selection, we did not ever need to eliminate a variable, but at each step, R did consider what the resulting AIC would be if one of the current variables was dropped (this is indicated by the - sign in the stepAIC table printout).

In [13]:
##f
y = hitters['Salary']
X = hitters.drop(['Salary', 'League', 'Division', 'NewLeague'], axis = 1)
league = pd.get_dummies(hitters['League'])
division = pd.get_dummies(hitters['Division'])
new_league = pd.get_dummies(hitters['NewLeague'])
X = pd.concat([X, league['A'], division['E'], new_league['A']], axis = 1)
X.columns = ['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists', 'Errors', 'League', 'Division', 'NewLeague']

## Cross Validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)

mod_full = smf.ols('Salary ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + PutOuts + Assists + Errors + League + Division + NewLeague', data = train).fit()
pred_full = mod_full.predict(X_test)
RMSE1_full = np.sqrt(np.mean((y_test - pred_full)**2))
RMSE1_full

374.1016115572486

In [14]:
m_optim = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + League + Division', data = train).fit()
pred_optim = m_optim.predict(X_test)
RMSE2_optim = np.sqrt(np.mean((y_test - pred_optim)**2))
RMSE2_optim

368.37302173645344

In [15]:
m_best_step = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division', data = train).fit()
pred_best_step = m_best_step.predict(X_test)
RMSE_best_step = np.sqrt(np.mean((y_test - pred_best_step)**2))
RMSE_best_step

365.15407981074935

***Note here that I did not need to find the BIC myself. Prof. Stevens told us what those variables were, but my calculations are confirmation, I guess :)

In [16]:
%%R
##We must use R with the bic on all poss summ
##we want to minimize the -logL
min_idxBIC <- which.min(all_poss_summ$bic)
min_idxBIC


Error in which.min(all_poss_summ$bic) : object 'all_poss_summ' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in which.min(all_poss_summ$bic) : object 'all_poss_summ' not found

  warnings.warn(x, RRuntimeWarning)


In [17]:
%%R
plot(all_poss_summ$bic, type = "l", xlab = "Number of Explanatory Variables", ylab = bquote("BIC"), main = "Optimal Number of Explanatory Variables")
points(all_poss_summ$bic, pch = 16)
points(x = min_idxBIC, y = all_poss_summ$bic[min_idxBIC], pch = 16, col = "red")


Error in plot(all_poss_summ$bic, type = "l", xlab = "Number of Explanatory Variables",  : 
  object 'all_poss_summ' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in plot(all_poss_summ$bic, type = "l", xlab = "Number of Explanatory Variables",  : 
  object 'all_poss_summ' not found

  warnings.warn(x, RRuntimeWarning)


In [18]:
%%R
##This tells us we want the following 6 variables in the model
all_poss_summ$which[min_idxBIC,]


Error in withVisible({ : object 'all_poss_summ' not found


It looks like AtBat, Hits, Walks, CRBI, Division, and PutOuts are the variables we should have in the model.

In [19]:
### 
m_best_BIC = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + PutOuts + Division', data = train).fit()
pred_best_BIC = m_best_BIC.predict(X_test)
RMSE3_best_BIC = np.sqrt(np.mean((y_test - pred_best_BIC)**2))
RMSE3_best_BIC

357.8142643379224

In [20]:
%%R
##part f
library(ISLR)
trn <- sample(x = c(rep(TRUE, round(0.8 * n)), rep(FALSE, n-round(0.8*n))), size = n, replace = FALSE)
train <- hitters[trn,]
tst <- !trn 
test <- hitters[tst,]

mFull <- lm(Salary ~ ., data = train)
predFull <- predict(object = mFull, newdata = test)
RMSEFull <- sqrt(mean((test$Salary - predFull)^2))
RMSEFull


Error in library(ISLR) : there is no package called ‘ISLR’


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in library(ISLR) : there is no package called ‘ISLR’

  warnings.warn(x, RRuntimeWarning)


In [21]:
%%R

mOptim <- lm(Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division + League, data = train)
predOptim <- predict(object = mOptim, newdata = test)
RMSEOptim <- sqrt(mean((test$Salary - predOptim)^2))
RMSEOptim


Error in is.data.frame(data) : object 'train' not found


/usr/local/lib/python2.7/dist-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in is.data.frame(data) : object 'train' not found

  warnings.warn(x, RRuntimeWarning)


In [22]:
%%R

mStep <- lm(Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division, data = train)
predStep <- predict(object = mStep, newdata = test)
RMSEStep <- sqrt(mean((test$Salary - predStep)^2))
RMSEStep


Error in is.data.frame(data) : object 'train' not found


In [23]:
%%R

mBIC <- lm(Salary ~ AtBat + Hits + Walks + CRBI + PutOuts + Division, data = train)
predBIC <- predict(object = mBIC, newdata = test)
RMSEBIC <- sqrt(mean((test$Salary - predBIC)^2))
RMSEBIC


Error in is.data.frame(data) : object 'train' not found


It's honestly really hard to tell which is the best using plain cross validation. The full model is always the worst, but the other three seem to take turns having the lowest RMSE. The values fluctuate pretty wildly with different random sets of train and test (from 210 ish to 400 some), and there is not one model that consistently has the lowest RMSE.

In [24]:
#g
## K-fold Cross Validation
#full
numfolds = 10
kf = KFold(n_splits=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf.split(X):
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + PutOuts + Assists + Errors + League + Division + NewLeague', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE1 = np.sqrt(MSE/numfolds)
RMSE1

358.0409538503632

In [25]:
#optim
numfolds = 10
kf = KFold(n_splits=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf.split(X):
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division + League', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE2 = np.sqrt(MSE/numfolds)
RMSE2

327.48898106270093

In [26]:
#step
numfolds = 10
kf = KFold(n_splits=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf.split(X):
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE3 = np.sqrt(MSE/numfolds)
RMSE3

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


328.95724975404283

In [27]:
#bic
numfolds = 10
kf = KFold(n_splits=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf.split(X):
    train_X = X.ix[train_indices, :]; train_y = y[train_indices]
    test_X = X.ix[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + PutOuts + Division', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE4 = np.sqrt(MSE/numfolds)
RMSE4

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


334.16342071770106

In [28]:
%%R

#g
## K-fold Cross-Validation
#full
library(boot)
m1 <- glm(Salary ~ ., data = hitters)
RMSE1 <- sqrt((cv.glm(hitters, m1, K = 10)$delta)[1])
RMSE1


Error in is.data.frame(data) : object 'hitters' not found


In [29]:
%%R
#optim
m2 <- glm(Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division + League, data = hitters)
RMSE2 <- sqrt((cv.glm(hitters, m2, K = 10)$delta)[1])
RMSE2


Error in is.data.frame(data) : object 'hitters' not found


In [30]:
%%R
#step
m3 <- glm(Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + PutOuts + Assists + Division, data = hitters)
RMSE3 <- sqrt((cv.glm(hitters, m3, K = 10)$delta)[1])
RMSE3


Error in is.data.frame(data) : object 'hitters' not found


In [31]:
%%R
#bic
m4 <- glm(Salary ~ AtBat + Hits + Walks + CRBI + PutOuts + Division, data = hitters)
RMSE4 <- sqrt((cv.glm(hitters, m4, K = 10)$delta)[1])
RMSE4


Error in is.data.frame(data) : object 'hitters' not found


Stepwise Selection Model appears to be the best. The K-Fold Validation method shows the Stepwise RMSE as about 0.5 (for an average random generation of training/test sets) less than that of the Optimal model. Since RMSE is a measure of how close the observed values are to the model's predicted values, a low RMSE indicates the model will have a good predictive accuracy.

h)

K-Fold Validation is much more reliable than Cross Validation (due to the potentially large variability of the latter), and it actually gives us consistent evidence that Stepwise Selection Model is the best choice for us to use. The K-fold validation estimates of RMSE are more stable since we take multiple measurements of the MSE and average them, so we can trust K-fold more and believe they more accurately depict the predictive accuracy of the model.

i)

I would probably want to use the Stepwise Selection Model if I were going for predictive accuracy. Even though SS Model has fewer variables than Optim Model, the one additional variable in Optim (League) must not be important enough to actually make a difference in the predictive accuracy. Also, the Stepwise Selection process showed that adding League to the model would raise the AIC; AIC quantifies the goodness of fit of the model while penalizing unneeded complexity, so the lower AIC of the SS Model (without League) as compared to the higher AIC of the model with League (Optim) tells us that adding League is more of a burden than a help in terms of simplicity and calculations.